연습문제
목표가 설정
아래 공식을 사용하여 목표가(Target)를 계산합니다.
목표가 = 당일 시가 + (전일 변동폭 * K)
변동폭 = 전일 고가 - 전일 저가
매수 조건
장중 고가가 목표가 이상일 경우 매수.
K값 수익률 분석
다양한 K값(예: 0.1 ~ 1.0)을 사용해 각 K값에 따른 전략 수익률을 계산하세요.
최적 K값 선택
가장 높은 누적 수익률(Cumulative Return)을 기록한 K값을 구하세요.





In [2]:
import pandas as pd 
from pykrx import stock # 9장에 사용법 나옴

In [3]:
df_kosdaq_150 = stock.get_market_ohlcv_by_date('20101001','20241230','229200')
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN
2015-10-02,10135,10159,10135,10148,63224,0.465300
2015-10-05,10255,10265,10210,10231,43154,0.817895
2015-10-06,10248,10274,10172,10176,42399,-0.537582
2015-10-07,10094,10094,9947,9949,21324,-2.230739


In [4]:
df_kosdaq_150['변동폭'] = df_kosdaq_150['고가'] - df_kosdaq_150['저가']
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭
날짜,,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN,67
2015-10-02,10135,10159,10135,10148,63224,0.465300,24
2015-10-05,10255,10265,10210,10231,43154,0.817895,55
2015-10-06,10248,10274,10172,10176,42399,-0.537582,102
2015-10-07,10094,10094,9947,9949,21324,-2.230739,147


In [5]:
df_kosdaq_150['전일변동폭'] = df_kosdaq_150['변동폭'].shift(periods=1) #전일 변동폭이라서 
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭
날짜,,,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN,67,NaN
2015-10-02,10135,10159,10135,10148,63224,0.465300,24,67.0
2015-10-05,10255,10265,10210,10231,43154,0.817895,55,24.0
2015-10-06,10248,10274,10172,10176,42399,-0.537582,102,55.0
2015-10-07,10094,10094,9947,9949,21324,-2.230739,147,102.0


In [6]:
K = 1/2
df_kosdaq_150['목표가'] = df_kosdaq_150['시가'] + (df_kosdaq_150['전일변동폭'] * K)
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭,목표가
날짜,,,,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN,67,NaN,NaN
2015-10-02,10135,10159,10135,10148,63224,0.465300,24,67.0,10168.5
2015-10-05,10255,10265,10210,10231,43154,0.817895,55,24.0,10267.0
2015-10-06,10248,10274,10172,10176,42399,-0.537582,102,55.0,10275.5
2015-10-07,10094,10094,9947,9949,21324,-2.230739,147,102.0,10145.0


In [7]:
conditions = df_kosdaq_150['고가']>= df_kosdaq_150['목표가']  # 매도 조건

In [8]:
import numpy as np
df_kosdaq_150['수익률'] = np.where(conditions, df_kosdaq_150['종가'] / df_kosdaq_150['목표가'], 1)

In [14]:
delta_days = df_kosdaq_150.index[-1] - df_kosdaq_150.index[0]
year = delta_days.days / 365

CAGR_dict = {}
for k in range(1,11):
    df_kosdaq_150['목표가'] = df_kosdaq_150['시가'] + (df_kosdaq_150['전일변동폭'] * (k/10))
    df_kosdaq_150['수익률'] = np.where(conditions, df_kosdaq_150['종가'] / df_kosdaq_150['목표가'], 1)
    df_kosdaq_150['누적수익률'] = df_kosdaq_150['수익률'].cumprod()
    CAGR = df_kosdaq_150['누적수익률'].iloc[-1] ** (1/year) -1
    CAGR_dict[f'{(k/10)}'] = CAGR * 100


CAGR_dict

{'0.1': 129.0180259097315,
 '0.2': 91.11210420386524,
 '0.3': 59.54876090624819,
 '0.4': 33.25520291630297,
 '0.5': 11.342049794983922,
 '0.6': -6.928360431694259,
 '0.7': -22.168130142382168,
 '0.8': -34.885402671325814,
 '0.9': -45.50220727754565,
 '1.0': -54.36920841121915}